In [ ]:
!pip install cirq
!pip install xlrd
!pip install pysf
!pip install openfermion
!pip install openfermionpyscf
import numpy as np

In [2]:
import cirq
import scipy
import pyscf
import openfermion
import openfermionpyscf 
import itertools

from pyscf import gto, scf, fci
from openfermion.transforms import jordan_wigner, get_fermion_operator
from openfermionpyscf import run_pyscf
from openfermion.utils import count_qubits
from openfermion.linalg import jw_hartree_fock_state
from openfermion.circuits import simulate_trotter
from openfermion import (
    get_sparse_operator, get_ground_state, FermionOperator,
    jw_get_ground_state_at_particle_number, MolecularData,
    expectation, uccsd_convert_amplitude_format,
    get_interaction_operator, QubitOperator, eigenspectrum,
    InteractionOperator
)

## Function Definitions

In [3]:
chemicalAccuracy = 1.5936*10**-3

# Define necessary Pauli operators (two-dimensional) as matrices
pauliX = np.array([[0,1],
                 [1,0]],
                dtype = complex)
pauliZ = np.array([[1,0],
                 [0,-1]],
                dtype = complex)
pauliY = np.array([[0,-1j],
                 [1j,0]],
                dtype = complex)

def stringToMatrix(pauliString):
  '''
  Converts a Pauli string to its matrix form.

  Arguments:
    pauliString (str): the Pauli string (e.g. "IXYIZ")

  Returns:
    matrix (np.ndarray): the corresponding matrix, in the computational basis

  '''

  matrix = np.array([1])

  # Iteratively construct the matrix, going through each single qubit Pauli term
  for pauli in pauliString:
      if pauli == "I":
        matrix = np.kron(matrix,np.identity(2))
      elif pauli == "X":
        matrix = np.kron(matrix,pauliX)
      elif pauli == "Y":
        matrix = np.kron(matrix,pauliY)
      elif pauli == "Z":
        matrix = np.kron(matrix,pauliZ)

  return matrix
  
def fromVectortoKet(stateVector):
  '''
  Transforms a vector representing a basis state to the corresponding ket.

  Arguments:
    stateVector (np.ndarray): basis vector in the 2^n-dimensional Hilbert space

  Returns:
    ket (list): a list of length n representing the corresponding ket 
  '''

  dim = len(stateVector)
  ket = []

  while dim>1:
    if any (stateVector[i] for i in range(int(dim/2))):
      # Ket is of the form |0>|...>. 

      #Fix |0> as the msq.
      ket.append(0)

      # Get the vector representing the state of the remaining qubits.
      stateVector = stateVector[:int(dim/2)]

    else:
      # Ket is of the form |1>|...>. 
      
      #Fix |0> as the msq.
      ket.append(1)

      # Get the vector representing the state of the remaining qubits.
      stateVector = stateVector[int(dim//2):]

    dim = dim/2

  return ket

def fromKettoVector(ket):
  '''
  Transforms a ket representing a basis state to the corresponding state vector.

  Arguments:
    ket (list): a list of length n representing the ket 

  Returns:
    stateVector (np.ndarray): the corresponding basis vector in the 
      2^n dimensional Hilbert space
  '''
  stateVector = [1]

  # Iterate through the ket, calculating the tensor product of the qubit states
  for i in ket:
    qubitVector = [not i,i]
    stateVector = np.kron(stateVector,qubitVector)

  return stateVector

def calculateOverlap(stateCoordinates1,stateCoordinates2):
    '''
    Calculates the overlap between two states, given their coordinates.

    Arguments:
      stateCoordinates1 (np.ndarray): the coordinates of one of the states in 
        some orthonormal basis,
      stateCoordinates2 (np.ndarray): the coordinates of the other state, in 
        the same basis

    Returns: 
      overlap (float): the overlap between two states (absolute value of the 
        inner product).
    '''

    bra = np.conj(stateCoordinates1)
    ket = stateCoordinates2
    overlap = np.abs(np.dot(bra,ket))
    
    return overlap

def findSubStrings(mainString,hamiltonian,checked = []):
    '''
    Finds and groups all the strings in a Hamiltonian that only differ from 
    mainString by identity operators.

    Arguments:
      mainString (str): a Pauli string (e.g. "XZ)
      hamiltonian (dict): a Hamiltonian (with Pauli strings as keys and their 
        coefficients as values)
      checked (list): a list of the strings in the Hamiltonian that have already
        been inserted in another group

    Returns: 
      groupedOperators (dict): a dictionary whose keys are boolean strings 
        representing substrings of the mainString (e.g. if mainString = "XZ", 
        "IZ" would be represented as "01"). It includes all the strings in the 
        hamiltonian that can be written in this form (because they only differ 
        from mainString by identities), except for those that were in checked
        (because they are already part of another group of strings).
      checked (list):  the same list passed as an argument, with extra values
        (the strings that were grouped in this function call).
    '''
    
    groupedOperators = {}
    
    # Go through the keys in the dictionary representing the Hamiltonian that 
    #haven't been grouped yet, and find those that only differ from mainString 
    #by identities
    for pauliString in hamiltonian:
        
        if pauliString not in checked:
            # The string hasn't been grouped yet
            
            if(all((op1 == op2 or op2 == "I") \
                   for op1,op2 in zip(mainString,pauliString))):
                # The string only differs from mainString by identities
                
                # Represent the string as a substring of the main one
                booleanString = "".join([str(int(op1 == op2)) for op1,op2 in \
                                       zip(mainString,pauliString)])
                    
                # Add the boolean string representing this string as a key to 
                #the dictionary of grouped operators, and associate its 
                #coefficient as its value
                groupedOperators[booleanString] = hamiltonian[pauliString]
                
                # Mark the string as grouped, so that it's not added to any 
                #other group
                checked.append(pauliString)
                
    return (groupedOperators,checked)

def groupHamiltonian(hamiltonian):
    '''
    Organizes a Hamiltonian into groups where strings only differ from 
    identities, so that the expectation values of all the strings in each 
    group can be calculated from the same measurement array.

    Arguments: 
      hamiltonian (dict): a dictionary representing a Hamiltonian, with Pauli 
        strings as keys and their coefficients as values.

    Returns: 
      groupedHamiltonian (dict): a dictionary of subhamiltonians, each of 
        which includes Pauli strings that only differ from each other by 
        identities. 
        The keys of groupedHamiltonian are the main strings of each group: the 
        ones with least identity terms. The value associated to a main string is 
        a dictionary, whose keys are boolean strings representing substrings of 
        the respective main string (with 1 where the Pauli is the same, and 0
        where it's identity instead). The values are their coefficients.
    '''
    groupedHamiltonian = {}
    checked = []
    
    # Go through the hamiltonian, starting by the terms that have less
    #identity operators
    for mainString in \
        sorted(hamiltonian,key = lambda pauliString: pauliString.count("I")):
            
        # Call findSubStrings to find all the strings in the dictionary that 
        #only differ from mainString by identities, and organize them as a 
        #dictionary (groupedOperators)
        groupedOperators,checked = findSubStrings(mainString,hamiltonian,checked)
        
        # Use the dictionary as a value for the mainString key in the 
        #groupedHamiltonian dictionary
        groupedHamiltonian[mainString] = groupedOperators
        
        # If all the strings have been grouped, exit the for cycle
        if(len(checked) == len(hamiltonian.keys())):
           break
       
    return groupedHamiltonian 
    
def convertHamiltonian(openfermionHamiltonian):
  '''
  Formats a qubit Hamiltonian obtained from openfermion, so that it's a suitable
  argument for functions such as measureExpectationEstimation.

  Arguments:
    openfermionHamiltonian (openfermion.qubitOperator): the Hamiltonian.

  Returns:
    formattedHamiltonian (dict): the Hamiltonian as a dictionary with Pauli
      strings (eg 'YXZI') as keys and their coefficients as values.
  '''

  formattedHamiltonian = {}
  qubitNumber = count_qubits(openfermionHamiltonian)

  # Iterate through the terms in the Hamiltonian
  for term in openfermionHamiltonian.get_operators():

    operators = []
    coefficient = list(term.terms.values())[0]
    pauliString = list(term.terms.keys())[0]
    previousQubit = -1

    for (qubit,operator) in pauliString:

      # If there are qubits in which no operations are performed, add identities 
      #as necessary, to make sure that the length of the string will match the 
      #number of qubits
      identities = (qubit-previousQubit-1)
      if identities>0: 
        operators.append('I'*identities)

      operators.append(operator)
      previousQubit = qubit
    
    # Add final identity operators if the string still doesn't have the 
    #correct length (because no operations are performed in the last qubits)
    operators.append('I'*(qubitNumber-previousQubit-1))

    formattedHamiltonian["".join(operators)] = coefficient

  return formattedHamiltonian

def hamiltonianToMatrix(hamiltonian):
    '''
    Convert a Hamiltonian (from OpenFermion) to matrix form.
    
    Arguments:
      hamiltonian (openfermion.InteractionOperator): the Hamiltonian to be
        transformed.

    Returns:
      matrix (np.ndarray): the Hamiltonian, as a matrix in the computational 
        basis
    
    ''' 
    
    qubitNumber = hamiltonian.n_qubits
    
    hamiltonian = jordan_wigner(hamiltonian)
    formattedHamiltonian = convertHamiltonian(hamiltonian)
    groupedHamiltonian = groupHamiltonian(formattedHamiltonian)

    matrix = np.zeros((2**qubitNumber,2**qubitNumber),dtype = complex)

    # Iterate through the strings in the Hamiltonian, adding the respective 
    #contribution to the matrix
    for string in groupedHamiltonian:

      for substring in groupedHamiltonian[string]:
        pauli = ("".join("I"*(not int(b)) + a*int(b) \
                         for (a,b) in zip(string,substring)))
        
        matrix += stringToMatrix(pauli) * groupedHamiltonian[string][substring]

    return matrix

def groundStatesFromDiagonalization(listOfHamiltonians):
    '''
    Obtains the ground states and energies corresponding to a list of 
    Hamiltonians, by performing exact diagonalization.

    Arguments:
      listOfHamiltonians (list): the list of hamiltonians as dictionaries.

    Returns: 
      exactGroundEnergies (list): the list of ground energies corresponding to
        the list of Hamiltonians
      exactGroundStates (list): the list of the ground states corresponding to 
        the list of hamiltonians
    diagonalization.
    '''
    
    exactGroundEnergies = []
    exactGroundStates = []
    
    # Obtain the number of qubits and calculate the dimension.
    n = len(list(listOfHamiltonians[0].keys())[0])
    dim = n**2

    # Find the ground state and energy of each Hamiltonian in the list.
    for hamiltonian in listOfHamiltonians:
        
        # Get the matrix form of the Hamiltonian, in the computational basis.
        hamiltonianMatrix = np.zeros((dim,dim),dtype = complex)

        for pauliString in hamiltonian:
            coefficient = hamiltonian[pauliString]
            hamiltonianMatrix += coefficient * stringToMatrix(pauliString)
            
        w,v = np.linalg.eigh(hamiltonianMatrix)
        
        # Add the minimum eigenvalue of the Hamiltonian to the list of ground
        #energies
        exactGroundEnergies.append((np.amin(w)))

        # Add the corresponging eigenstate to the list of ground states
        exactGroundStates.append(v[np.argmin(w)])
        
    return (exactGroundEnergies,exactGroundStates)

def unitaryToRotationGates(unitaryMatrix,qubit,keepPhase = False):
    ''' 
    Decomposes a single qubit unitary matrix into rotation gates.

    Arguments: 
      unitaryMatrix (np.ndarray): the matrix to be decomposed
      qubit (cirq.LineQubit): the qubit the matrix acts on
      keepPhase (bool): a flag indicating whether the global phase should be 
        kept

    Returns: 
      (list): a list of single qubit rotation gates, equivalent to the action of 
        unitaryMatrix (up to a global phase, if keepPhase = False)
    ''' 
    
    # Compute the angles of each rotation from the entries of the matrix
    rz1angle = -np.angle(unitaryMatrix[0][0])+np.angle(-unitaryMatrix[0][1])
    ryangle = 2*np.arccos(np.abs(unitaryMatrix[0][0]))
    rz2angle = -np.angle(unitaryMatrix[0][0])+np.angle(unitaryMatrix[1][0])
    
    if keepPhase:
        gp = np.angle(unitaryMatrix[0][0])+(rz1angle+rz2angle)/2
    
    # Create the necessary rotation gates
    r1 = cirq.rz(rz1angle)
    r2 = cirq.ry(ryangle)
    r3 = cirq.rz(rz2angle)
    
    yield [r1(qubit),r2(qubit),r3(qubit)]

    if keepPhase:
        # The global phase should be kept; add the correction.
        yield cirq.MatrixGate(np.identity(2)*np.exp(1j*gp)).on(qubit)

def measureString(pauliString,repetitions,statePreparationGates,qubits):
    ''' 
    Measures the expectation value of a Pauli string using the CIRQ simulator 
    (simulating sampling).

    Arguments:
      pauliString (str): the Pauli string to be measured.
      repetitions (int): the number of circuit repetitions to be used.
      statePreparationGates (list): the list of CIRQ gates that prepare the 
        state in which to obtain the expectation value.

    Returns: 
      expecationValue (float): the expectation value of pauliString, with 
        sampling noise.
    ''' 

    # Initialize circuit.
    circuit = cirq.Circuit(statePreparationGates)

    # Optimize circuit.
    cirq.optimizers.EjectZ().optimize_circuit(circuit)
    cirq.optimizers.DropNegligible().optimize_circuit(circuit)
    
    # Append necessary rotations and measurements for each qubit.
    for i in range(len(qubits)):
        op = pauliString[i]
        
        # Rotate qubit i to the X basis if that's the desired measurement.
        if (op == "X"):
            circuit.append(cirq.H(qubits[i]))
        
        # Rotate qubit i to the Y basis if that's the desired measurement.
        if (op == "Y"):
            # Apply adjoint of the Clifford S gate
            circuit.append(cirq.ops.ZPowGate(exponent = -1/2).on(qubits[i]))

            # Finish rotating to Y basis, proceeding as for X
            circuit.append(cirq.H(qubits[i]))
            
        # Measure qubit i in the computational basis, unless operator is I.
        if (op != "I"):
            circuit.append(cirq.measure(qubits[i],key = str(i)))
            
    # Sample the desired number of repetitions from the circuit, unless
    #there are no measurements (identity term).
    if (pauliString != "I"*len(qubits)):
        s = cirq.Simulator()
        results = s.run(circuit,repetitions = repetitions)
    #print(circuit)

    # Calculate the expectation value of the Pauli string by averaging over  
    #all the repetitions.
    
    total = 0
    
    for j in range(repetitions):
        meas = 1
        for i in range(len(qubits)):
            if (pauliString[i] != "I"):
                meas = meas*(1-2*results.data[str(i)][j])
        total += meas
        
    expectationValue = total/repetitions
    
    return expectationValue

def measureExpectation(mainString,subHamiltonian,repetitions,\
                       statePreparationGates,qubits):
    ''' 
    Measures the expectation value of a subHamiltonian using the CIRQ simulator 
    (simulating sampling). By construction, all the expectation values of the 
    strings in subHamiltonian can be obtained from the same measurement array.

    Arguments: 
      mainString (str): the main Pauli string. This is the string in the group
        with the least identity terms. It defines the circuit that will be used.
      subHamiltonian (dict): a dictionary whose keys are boolean strings
        representing substrings of the main one, and whose values are the 
        respective coefficients.
      repetitions (int): the number of repetitions to be performed, the 
      statePreparationGates (list): the list of CIRQ gates that prepare (from 
        |0..0>) the state in which to obtain the expectation value.
      qubits (list): list of cirq.LineQubit to apply the gates on

    Returns:
      totalExpectationValue (float): the total expectation value of 
        subHamiltonian, with sampling noise.
    ''' 
    
    # Initialize circuit.
    circuit = cirq.Circuit()
    
    # Append to the circuit the gates that prepare the state corresponding to
    #the received parameters.
    circuit.append(statePreparationGates)
    cirq.optimizers.EjectZ().optimize_circuit(circuit)
    cirq.optimizers.DropNegligible().optimize_circuit(circuit)
    
    # Append necessary rotations and measurements for each qubit.
    for i in range(len(qubits)):
        op = mainString[i]
        
        # Rotate qubit i to the X basis if that's the desired measurement.
        if (op == "X"):
            circuit.append(cirq.H(qubits[i]))
            
        # Rotate qubit i to the X basis if that's the desired measurement.
        if (op == "Y"):
            # Apply adjoint of the Clifford S gate
            circuit.append(cirq.ops.ZPowGate(exponent = -1/2).on(qubits[i]))

            # Apply the Hadamard gate
            circuit.append(cirq.H(qubits[i]))
            
        #Measure qubit i in the computational basis, unless operator is I.
        if (op != "I"):
            circuit.append(cirq.measure(qubits[i],key = str(i)))
            
    # Sample the desired number of repetitions from the circuit, unless
    #there are no measurements (identity term).
    if (mainString != "I"*len(qubits)):
        s = cirq.Simulator()
        results = s.run(circuit,repetitions = repetitions)

    # For each substring, initialize the sum of all measurements as zero
    total = {}
    for subString in subHamiltonian:
        total[subString] = 0
    
    # Calculate the expectation value of each Pauli string by averaging over  
    #all the repetitions
    for j in range(repetitions):
        meas = {}
        
        # Initialize the measurement in repetition j for all substrings
        for subString in subHamiltonian:
            meas[subString] = 1
        
        # Go through the measurements on all the qubits
        for i in range(len(qubits)):
            
            if (mainString[i] != "I"):
                # There's a measurement associated with this qubit
                
                # Use this single qubit measurement for the calculation of the
                #measurement of each full substring in this repetition. If the
                #substring has a "0" in the position corresponding to this
                #qubit, the operator associated is I, and the measurement
                #is ignored (raised to the power of 0)
                for subString in subHamiltonian:
                    meas[subString] = meas[subString]*((1-2*results.data[str(i)][j])\
                                                     **int(subString[i]))
                        
        # Add this measurement to the total, for each string
        for subString in subHamiltonian:
            total[subString]+=meas[subString]
        
    totalExpectationValue = 0
    
    # Calculate the expectation value of the subHamiltonian, by multiplying
    #the expectation value of each substring by the respective coefficient
    for subString in subHamiltonian:
        
        # Get the expectation value of this substring by taking the average
        #over all the repetitions
        expectationValue = total[subString]/repetitions
        
        # Add this value to the total expectation value, weighed by its 
        #coefficient
        totalExpectationValue+=expectationValue*subHamiltonian[subString]
    
    return(totalExpectationValue)

count = 0
trackOptimization = False

def stateEnergy(statePreparationGates,hamiltonian,repetitions,qubits):
    ''' 
    Returns: the experimental energy expectation in a given state.
    Arguments: a list of 6 parameters defining the state in which to obtain the 
    expectation value, a dictionary with the coefficients of each Pauli string
    term in the Hamiltonian, the number of repetitions to be used to calculate 
    the expectation value of each string.
    ''' 
    
    # Print the percentage of the maximum number of function evaluations that 
    #has been used so far in the classical optimization, if it's a multiple of
    #10% (to inform on the stage of the optimization process).
    global count, trackOptimization
    if(trackOptimization):
        if (count%(300/10) == 0):
            print(round(count/(300/100)),"%",sep = '')
        count = count+1

    groupedHamiltonian = groupHamiltonian(hamiltonian)

    experimentalEnergyExpectation = 0
    
    # Obtain experimental expectation value for each necessary Pauli string by
    #calling the measureExpectation function, and perform the necessary weighed
    #sum to obtain the energy expectation value.
    for mainString in groupedHamiltonian:
         
        expectationValue = measureExpectation(mainString,\
                                            groupedHamiltonian[mainString],\
                                                repetitions,statePreparationGates,qubits)
        experimentalEnergyExpectation+=expectationValue

    return experimentalEnergyExpectation

def exactStateEnergy(stateCoordinates,hamiltonian):
    ''' 
    Calculates the exact energy in a specific state.

    Arguments:
      stateCoordinates (np.ndarray): the state in which to obtain the 
        expectation value.
      hamiltonian (dict): the Hamiltonian of the system.
    
    Returns:
      exactEnergy (float): the energy expecation value in the state.
    ''' 

    exactEnergy = 0
    
    # Obtain the theoretical expectation value for each Pauli string in the
    #Hamiltonian by matrix multiplication, and perform the necessary weighed
    #sum to obtain the energy expectation value.
    for pauliString in hamiltonian:
        
        ket = np.array(stateCoordinates,dtype = complex)
        bra = np.conj(ket)
        
        ket = np.matmul(stringToMatrix(pauliString),ket)
        expectationValue = np.real(np.dot(bra,ket))
        
        exactEnergy+=\
            hamiltonian[pauliString]*expectationValue
            
    return exactEnergy
    
def exactStateEnergySparse(stateVector,sparseHamiltonian):
    ''' 
    Calculates the exact energy in a specific state, using sparse matrices.

    Arguments:
      stateVector (Union[np.ndarray, scipy.sparse.csc_matrix): the state in 
        which to obtain the expectation value.
      sparseHamiltonian (scipy.sparse.csc_matrix): the Hamiltonian of the system.
    
    Returns:
      energy (float): the energy expecation value in the state.
    ''' 

    if not isinstance(stateVector,scipy.sparse.csc_matrix):
      ket = scipy.sparse.csc_matrix(stateVector,dtype=complex).transpose()
    else:
      ket = stateVector
      
    bra = ket.transpose().conj()

    energy = (bra * sparseHamiltonian * ket)[0,0].real
    
    return energy
    
def energyFromExpOperator(operator,referenceState,sparseHamiltonian):
  '''
  Calculates the energy in the state obtained by applying e^operator to the
  reference state, using sparse matrices.

  Arguments:
    operator (openfermion.FermionOperator): the operator to be exponentiated
      and applied to the reference state.
    referenceState (np.ndarray): the reference state.
    sparseHamiltonian (scipy.sparse.csc_matrix): the Hamiltonian of the system,
      in matrix form.

  Returns:
    energy (float): the energy.
  '''
  # Obtain the dimension and, from it, calculate the number of qubits
  dimension, _ = sparseHamiltonian.shape
  qubitNumber = int(np.log(dimension)/np.log(2)) 
  
  # Obtain the sparse matrix representing the operator, and act on the state
  #with its exponentiated version
  sparseOperator = get_sparse_operator(operator,qubitNumber)
  ket = scipy.sparse.csc_matrix(referenceState,dtype=complex).transpose()
  ket = scipy.sparse.linalg.expm_multiply(sparseOperator,ket)

  # Calculate the energy expectation value
  bra = ket.transpose().conj()
  energy = (bra * sparseHamiltonian * ket)[0,0].real

  return energy

## Molecule Definitions 

In [4]:
# H2
geometry = [['H',[0,0,0]],['H',[0,0,0.74]]]
basis = 'sto-3g'
multiplicity = 1
charge = 0
h2molecule = MolecularData(geometry,basis,multiplicity,charge)
h2molecule = run_pyscf(h2molecule,run_fci = True,run_ccsd = True)

# HeH+
r = 1 # interatomic distance in angstrom
geometry = [['He',[0,0,0]],['H',[0,0,r]]]
basis = 'sto-3g'
multiplicity = 1
charge = +1
helonium = MolecularData(geometry,basis,multiplicity,charge)
helonium = run_pyscf(helonium,run_fci = True,run_ccsd = True)

# Get the PySCF molecule as well (it's not necessary as there is the OF-PySCF
#plugin, just for confirmation)
helonium_pysf = gto.M(atom = geometry, charge = charge, basis = basis,verbose = 0)

# HeH+
r = 1 # interatomic distance in angstrom
geometry = [['He',[0,0,0]],['H',[0,0,r]]]
basis = 'sto-3g'
multiplicity = 1
charge = +1
helonium = MolecularData(geometry,basis,multiplicity,charge)
helonium = run_pyscf(helonium,run_fci = True,run_ccsd = True)

# LiH
bondLength = 1.45 # interatomic distance in angstrom
geometry = [['Li',[0,0,0]],['H',[0,0,bondLength]]]
basis = 'sto-3g'
multiplicity = 1
charge = 0
liH = MolecularData(geometry,basis,multiplicity,charge)
liH = run_pyscf(liH,run_fci = True,run_ccsd = True)

# Alternative to using run_pyscf: load from OpenFermion (data for this 
#particular molecule at this particule interatomic distance is available in 
#a file that comes with OF)
liHOF = MolecularData(geometry,basis,multiplicity,charge,description = '1.45')
liHOF.load()

## Choosing the molecule

In [5]:
molecule = helonium

In [6]:
qubitNumber = molecule.n_qubits
electronNumber = molecule.n_electrons
orbitalNumber = molecule.n_orbitals

## Getting some data on the molecule

### Hamiltonian

The Hamiltonian of the molecule (in terms of fermionic raising/creation and lowering/annihilation operators) can be obtained using **get_molecular_hamiltonian**, a method of the **MolecularData** class. 

When no arguments are supplied, the method considers all orbitals active. Since the *STO-3G* minimal basis was chosen in setting the molecule, *all orbitals* means all the ones considered in a minimal basis: those that belong to a (fully or partially) occupied shell. 

>*Example*: In helonium, for instance, this is equivalent to setting >*active_indices = [0,1]*. Indices are of *spatial* orbitals. 

Orbital that belong to the same shell have the same principal quantum number. So if we have $_1H: 1s^1$ or $_2He: 1s^2$ we use one orbital ($1s$); if we have $_3Li:1s^2 1s^1$ we use 5 ($1s$, $2s$, $2p_x$, $2p_y$, $2p_z$).

*STO* in *STO-3G* stands for *Slater type orbitals*, which means that we're using as molecular orbitals (wavefunctions of a many-particle system) anti-symmetrized products of atomic orbitals (wavefunctions of the individual particles). *3G* means thate we're approximating each Slater determinant by three Gaussian orbitals.


In [7]:
hamiltonian = molecule.get_molecular_hamiltonian() 

The Hamiltonian is returned as an instance of **InteractionOperator**. This is a special class for storing fermionic operators that:

- Consist only of one-body and two-body terms
- Consists only of terms that preserve particle number and spin


In [8]:
print(type(hamiltonian))

<class 'openfermion.ops.representations.interaction_operator.InteractionOperator'>


The data structure consists of:
- A constant term (attribute **constant**)
- An NxN array of floats (matrix) representing the one body terms (attribute **one_body_tensor**)
- An NxNxNxN array of floats representing the two body terms (attribute **two_body_tensor**)

Where N is the number of qubits necessary to represent the molecule, that corresponds to the number of fermionic modes, or spin orbitals, that we choose to consider active.

As an example, if term $h_{p, q, r, s} a^\dagger_p a^\dagger_q a_r a_s$ occurs in *hamiltonian*, then *hamiltonian.two_body_tensor* [p,q,r,s] = $h_{p,q,r,s}$.

Since each term is particle number conserving, it must have as many creation operators as annihilation operators. So it's enough to decide a specific order for them to make the tensors non-ambiguous. The convention is *normal ordering* them. This corresponts to putting all creation operators to the left.

In [9]:
print("Constant:\n",hamiltonian.constant)
print("Shape of One Body Tensor:\n",hamiltonian.one_body_tensor.shape)
print("Shape of Two Body Tensor:\n",hamiltonian.two_body_tensor.shape)
print("Hamiltonian:")
print(hamiltonian)

Constant:
 1.05835442184
Shape of One Body Tensor:
 (4, 4)
Shape of Two Body Tensor:
 (4, 4, 4, 4)
Hamiltonian:
() 1.05835442184
((0, 1), (0, 0)) -2.4305217022828143
((0, 1), (2, 0)) 0.17523764451269103
((1, 1), (1, 0)) -2.4305217022828143
((1, 1), (3, 0)) 0.17523764451269103
((2, 1), (0, 0)) 0.1752376445126909
((2, 1), (2, 0)) -1.3235920553067362
((3, 1), (1, 0)) 0.1752376445126909
((3, 1), (3, 0)) -1.3235920553067362
((0, 1), (0, 1), (0, 0), (0, 0)) 0.4748839522005155
((0, 1), (0, 1), (0, 0), (2, 0)) -0.08761816148293655
((0, 1), (0, 1), (2, 0), (0, 0)) -0.08761816148293655
((0, 1), (0, 1), (2, 0), (2, 0)) 0.05857335548569318
((0, 1), (1, 1), (1, 0), (0, 0)) 0.4748839522005155
((0, 1), (1, 1), (1, 0), (2, 0)) -0.08761816148293655
((0, 1), (1, 1), (3, 0), (0, 0)) -0.08761816148293655
((0, 1), (1, 1), (3, 0), (2, 0)) 0.05857335548569318
((0, 1), (2, 1), (0, 0), (0, 0)) -0.08761816148293655
((0, 1), (2, 1), (0, 0), (2, 0)) 0.05857335548569318
((0, 1), (2, 1), (2, 0), (0, 0)) 0.285053192

It is necessary to represent the problem in a way that allows solving it with a quantum computer. Fermionic states will have to be mapped into qubit states, and fermionic operators will have to be mapped into qubit operators.

For example, it is necessary to transform the Hamiltonian into one that acts on the qubits that represent the fermionic state.

This is achieved by performing the **Jordan Wigner** transformation, which maps a **FermionOperator** into a **QubitOperator**. Since the function only accepts FermionOperators, the Hamiltonian must be transformed into one first. This is always possible, because instances of **InteractionOperator** are fermion operators with special characteristics.



In [10]:
qubitHamiltonian = jordan_wigner(get_fermion_operator(hamiltonian))

# Eliminate terms that are negligeable up to a certain tolerance
qubitHamiltonian.compress()

If we pass the qubit Hamiltonian as an argument to **count_qubits**, it will return the minimum number of qubits it acts on.  Under the Jordan Wigner transformation, each qubit represents a specific fermionic mode (spin orbital). 

The number will be the same as **molecule.n_qubits**: the number of qubits necessary to represent a state of the molecule.

>*Example*: 
In $HeH^+$, there are 4 spin orbitals: two $1s$ spatial orbitals from each atom, each with two associated spin orbitals ($\alpha$ and $\beta$ spins). 
>
>In order to represent a state of this molecule, we need four qubits: the state of each one will represent the occupation number of the corresponding spin orbital (0 or 1, given the Pauli exclusion principle).
>
>A qubit operator acting on the state has to live in the same dimension or in a lower one. As for the Hamiltonian, it necessarily involves all of the particles representing the molecule: if they didn't affect the energy, they wouldn't be in the molecule.

In [11]:
# Compute the minimum number of qubits on which operator acts; it's at most 
#molecule.n_qubits
print("Qubit Number:",count_qubits(qubitHamiltonian))

# Get the number of spatial orbitals of the molecule. 
# Since for each spacial orbital there are two spin orbitals (alpha, 'up', or
#beta, 'down', spins), this is half the number of qubits.
print("Orbital Number:",molecule.n_orbitals)

# Get the energies associated to the orbitals.
# Lower index -> lower energy.
print("Orbital Energies:",molecule.orbital_energies)

Qubit Number: 4
Orbital Number: 2
Orbital Energies: [-1.48075592 -0.30052523]


### Hartree Fock State

In [12]:
# Get the Hartree Fock state, assuming the Jordan Wigner mapping.
# Second argument corresponds to SPIN orbitals, hence the factor of two.
hfState = jw_hartree_fock_state(molecule.n_electrons,molecule.n_orbitals*2) 

print("Hartree Fock Vector:",hfState)

# The vector corresponds to a computational basis state with the qubits up to
#molecule.n_electrons in state |1>, and all the other ones in state |0> 
#(|11...00>).
# Corresponds to a single Slater Determinant, which includes the lowest energy
#atomic orbitals. 
# By acting on this state with excitation operators, the UCCSD ansatz will 
#create a superposition of Slater Determinants that may correspond to an 
#entangled state.
print("Hartree Fock Ket:",fromVectortoKet(hfState))

# So the Hartree Fock state preparation circuit is implemented on CIRQ as:
qubits = cirq.LineQubit.range(molecule.n_qubits)

hfGates = [cirq.X(qubits[i]) for i in range(molecule.n_electrons)]

# Which is equivalent to:
hfGates = [cirq.X(qubits[i]) for i in range(molecule.n_qubits) \
  if fromVectortoKet(hfState)[i]] 

Hartree Fock Vector: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Hartree Fock Ket: [1, 1, 0, 0]


### Ground Energy

In [13]:
# Get the sparse matrix representing the Hamiltonian in the qubit space
sparseHamiltonian = get_sparse_operator(hamiltonian)

# Reshape the qubit Hamiltonian into a dictionary associating to Pauli strings
#(e.g. 'XXYY') their coefficients (as suitable for use in 
#stateEnergy)
formattedHamiltonian = convertHamiltonian(qubitHamiltonian)

# Group strings that only differ by identity terms
groupedHamiltonian = groupHamiltonian(formattedHamiltonian)

# One can get the Hamiltonian's matrix form by adding the matrices corresponding
#to each of the Pauli strings, as 
#hamiltonianMatrix = hamiltonianToMatrix(hamiltonian)
#However, it's really inefficient; it's better to use the sparse matrix from
#OpenFermion

qubits = cirq.LineQubit.range(molecule.n_qubits)
shots = 1000

# Measure HF energy by getting the statistics on the CIRQ simulator
myenergy = stateEnergy(hfGates,formattedHamiltonian,shots,qubits)

# Calculate exact HF energy using matrix algebra
calculatedEnergy = exactStateEnergy(hfState,formattedHamiltonian) 

# For greater efficiency, it's better to use sparse matrices:
calculatedEnergySparse = exactStateEnergySparse(hfState,sparseHamiltonian)

tol = 10**-10
assert(calculatedEnergy-calculatedEnergySparse < tol)

# Get the energy from PySCF
hf_pyscf = scf.RHF(helonium_pysf)
hfEnergy_pyscf = hf_pyscf.kernel()

print("HF\nMy measured energy: {}".format(myenergy))

print("My calculated energy: {}".format(calculatedEnergy))

print("Openfermion energy (attribute): {}".format(molecule.hf_energy))

print("Openfermion energy (calculated): {}".format(np.real\
      (openfermion.expectation(get_sparse_operator(qubitHamiltonian),hfState))))

print("PySCF energy: {}".format(hfEnergy_pyscf))

error = molecule.hf_energy - molecule.fci_energy
print("\nError:",error)
print("(in % of chemical accuracy: {:.3f}%)\n".format(error/chemicalAccuracy*100))

HF
My measured energy: -2.853916883515915
My calculated energy: -2.852921078324599
Openfermion energy (attribute): -2.852921078324597
Openfermion energy (calculated): -2.852921078324599
PySCF energy: -2.852921078324597

Error: 0.0072840442552060125
(in % of chemical accuracy: 457.081%)



In [14]:
# Print the ground energy obtained from the classical variational full 
#configuration interaction approach (exact within the STO-3G basis set)
print("FCI Energy (OpenFermion): {}".format(molecule.fci_energy))

# Get the same energy from PySCF
cisolver = fci.FCI(helonium_pysf, hf_pyscf.mo_coeff)
fciEnergy_pysf = cisolver.kernel()[0]
print("FCI Energy (pyscf):",fciEnergy_pysf)

# Print the ground energy obtained from the classical variational coupled 
#cluster singles and doubles approach
print("CCSD Energy: {}\n".format(molecule.ccsd_energy))

# Get the ground state and energy of the Hamiltonian using OpenFermion's
#get_ground_state function, which resorts scipy.sparse.linalg.eigsh
# It's significantly lower than FCI.
energyFromSparseOperator, stateFromSparseOperator = \
  get_ground_state(sparseHamiltonian)

print("Energy from sparse operator: {}".format(energyFromSparseOperator))

calculatedEnergy = exactStateEnergySparse(stateFromSparseOperator,sparseHamiltonian) 

print("Same state, my calculated energy: {}".format(calculatedEnergy))

# Diagonalize Hamiltonian to get the exact ground state and energy, using my
#own function. Same energy.
formattedHamiltonian = convertHamiltonian(qubitHamiltonian)

(exactGroundEnergies,exactGroundStates) = groundStatesFromDiagonalization\
  ([formattedHamiltonian])

print("My energy from exact diagonalization: {}".format\
      (np.real(exactGroundEnergies[0])))

exactState = exactGroundStates[0]

# Calculate overlap of the exact ground state obtained from exact 
#diagonalization with the hartree-fock state. 
print("Overlap with HF state:",calculateOverlap(exactState,hfState))

# Get eigenvalues of the Hamiltonian. The FCI energy is there, but there's a 
#lower value. 
# There shouldn't be? The Slater determinant with the lowest energy isn't the
#one corresponding to the Hartree Fock solution, rather one one the wrong 
#number of electrons.
print("Eigenspectrum of the Hamiltonian:\n",eigenspectrum(qubitHamiltonian))

# The lower eigenvalue can't correspond to a physical state of the 
#system. The ground states obtained from get_ground_state or from my 
#diagonalization, that supposedly have lower energy than the FCI state, have 
#nonzero coefficients for e.g. |0001>. The real system should have always
#2 electrons. FCI operators are all particle number conserving, they can't
#reach a state with the wrong number of electrons starting from the Hartree Fock
#state, and that's a good thing. To get the ground state from the matrix 
#of the Hamiltonian, it's necessary to first restrict it to live in the 
#space spanned by computational basis states that have the correct number of 
#electrons. See  openfermion.linalg.sparse_tools.jw_number_restrict_operator

# Get all possible combinations of qubits to hold the correct number of 
#electrons. This is done by selecting electronNumber qubit indices, with no 
#repetition. 1 qubit in state |1> corresponds to 1 electron, because under 
#the Jordan Wigner mapping it represents the occupation of a specific spin
#orbital. For HeH+ we have two electrons, so we get all the different pairs
#of indices.
occupations = itertools.combinations(range(qubitNumber), electronNumber)

# Transform the selected qubit labels given by 'occupations' into indices.
# Example: the Hartree Fock state |1100> will have index 2^0 + 2^1 = 3.
select_indices = [sum([2**n for n in occupation]) for occupation in occupations]

# Construct a restricted version of the Hamiltonian, by selecting only those 
#lines and columns that correspond to computational basis states that have the 
#correct number of particles.
restrictedHamiltonian = sparseHamiltonian[np.ix_(select_indices, select_indices)]

energyFromSparseOperator, stateFromSparseOperator = \
  get_ground_state(restrictedHamiltonian)

print("\nEnergy from sparse operator, restricted to the correct particle number: ",\
      (energyFromSparseOperator))

# The length of the resulting ground state is not the dimension of the Hilbert
#space, but the dimension of the subspace spanned by the computational basis
#states that correspond to Slater Determinants with the correct number of 
#electrons.
# For HeH+ there are only 6 coefficients (full Hilbert space is of dimension 16),
#corresponding to the 6 computational basis states that two and only two |1> 
#states).
print("Length of restricted vector:",len(stateFromSparseOperator))

# None of this should be necessary and this only happens for HeH+. Also happens
#using Qiskit and Pyscf. Something to do with the representation of the energy
#in Pyscf?

FCI Energy (OpenFermion): -2.860205122579803
FCI Energy (pyscf): -2.860205122579803
CCSD Energy: -2.860205139888532

Energy from sparse operator: -3.1578592138000037
Same state, my calculated energy: -3.1578592138000006
My energy from exact diagonalization: -3.1578592138000006
Overlap with HF state: 0.0
Eigenspectrum of the Hamiltonian:
 [-3.15785921 -3.15785921 -2.86020512 -2.70774238 -2.2456425  -2.2456425
 -2.24279966 -2.24279966 -2.24279966 -2.0978611  -1.39924664 -1.39924664
 -0.74596009 -0.23815828 -0.23815828  1.05835442]

Energy from sparse operator, restricted to the correct particle number:  -2.8602051225798033
Length of restricted vector: 6
